<!-- # Compare Runs, Choose a Model, Deploy the model to a REST API -->

    -Run Hyperparameter Sweep on Training Script
    -Compare the Results of Runs on MLFlow UI
    -Choose the best Model and Register it as a Model
    -Deploy the Model on REST API
    -Build a Container Image suitable for Deployment in a Cloud Platform

In [73]:
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
import mlflow
from mlflow.models import infer_signature

data_df = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")
data_df.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [74]:
x_train = data_df.drop(columns='quality',axis=1).values
y_train = data_df['quality'].values.ravel()

y_test = data_df.drop(columns='quality',axis=1).values
x_test = data_df['quality'].values.ravel()

In [75]:
x_train,x_valid,y_train,y_valid = train_test_split(x_train,y_train,test_size=0.2,random_state=42)


In [76]:
#ANN Model

import mlflow.tensorflow
import mlflow.tensorflow


def train_model(params, epochs,  train_x, train_y, valid_x, valid_y, test_x, test_y):
    #Model Architecture
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)

    model = keras.Sequential(
        [
            keras.Input([x_train.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)
        ]
    )

    #Compile Model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"], momentum=params["momentum"]

    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    #Train ANN Model with LR and Momentum params with MLFlow Tracking 
    mlflow.set_experiment("Wine Quality: ANN Model Selection --ALL MODELS")
    with mlflow.start_run(nested=True):
        model.fit(x_train,y_train,validation_data=(x_valid,y_valid), epochs = epochs, batch_size=65)
        
        #Evaluate Model
        eval_result = model.evaluate(valid_x,valid_y,batch_size=65)
        eval_rmse = eval_result[1]

        
        #Log Parameters and Results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        #Log the Model
        from mlflow.models import infer_signature
        signature = infer_signature(train_x, model.predict(train_x[:5]))
        mlflow.keras.log_model(model, artifact_path="model", signature=signature)

        return {"loss" : eval_rmse, "status" : STATUS_OK, "model" : model }




In [77]:
def objective(params):
    #MLFlow will track the parameters and results for each run
     result = train_model(
          params,
          epochs=8,
          train_x=x_train,
          train_y=y_train,
          valid_x=x_valid,
          valid_y=y_valid,
          test_x=x_test,
          test_y=y_test
     )
     return result

In [78]:
space = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-2)),
    "momentum":hp.uniform("momentum",0.7,1)
    

}

In [79]:
import mlflow.keras
import mlflow.keras
import mlflow.tensorflow
import mlflow.tensorflow


mlflow.set_experiment("Wine Quality: ANN Model Selection --Best Models")
with mlflow.start_run():
    #HyperParameter Search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=25,
        trials=trials
    )
    

    #Details of Best Run
    best_run = sorted(trials.results,key=lambda x: x["loss"])[0]

    #Log Best Parameters, Loss, and Model
    mlflow.log_params(best)
    mlflow.log_metric("eval mse", best_run["loss"])
    from mlflow.models import infer_signature
    signature = infer_signature(x_train, best_run["model"].predict(x_train[:5]))
    mlflow.keras.log_model(best_run["model"], artifact_path="model", signature=signature)

    #Print best params
    print(f"Best parameters: {best}")
    print(f"Best eval rsme: {best_run["loss"]}")
    print(f"Worst eval rsme: {sorted(trials.results,key=lambda x: x["loss"])[-1]}")


2025/07/14 12:51:06 INFO mlflow.tracking.fluent: Experiment with name 'Wine Quality: ANN Model Selection --Best Models' does not exist. Creating a new experiment.


  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

2025/07/14 12:51:06 INFO mlflow.tracking.fluent: Experiment with name 'Wine Quality: ANN Model Selection --ALL MODELS' does not exist. Creating a new experiment.



Epoch 1/8                                             

 1/61 ━━━━━━━━━━━━━━━━━━━━ 32s 544ms/step - loss: 34.0211 - root_mean_squared_error: 5.8328
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 14.2999 - root_mean_squared_error: 3.6554 - val_loss: 1.9531 - val_root_mean_squared_error: 1.3975

Epoch 2/8                                             

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.4353 - root_mean_squared_error: 1.1980
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step - loss: 1.4457 - root_mean_squared_error: 1.2020 - val_loss: 1.4982 - val_root_mean_squared_error: 1.2240

Epoch 3/8                                             

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.4272 - root_mean_squared_error: 1.1946
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - loss: 1.1579 - root_mean_squared_error: 1.0759 - val_loss: 1.2462 - val_root_mean_squared_error: 1.1163

Epoch 4/8                                             

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.0758 - root_me

2025/07/14 12:51:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



...

Epoch 1/8                                                                       

 1/61 ━━━━━━━━━━━━━━━━━━━━ 22s 381ms/step - loss: 35.9269 - root_mean_squared_error: 5.9939
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 13.6548 - root_mean_squared_error: 3.5880 - val_loss: 2.0615 - val_root_mean_squared_error: 1.4358

Epoch 2/8                                                                       

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.8583 - root_mean_squared_error: 1.3632
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - loss: 1.3321 - root_mean_squared_error: 1.1511 - val_loss: 0.7728 - val_root_mean_squared_error: 0.8791

Epoch 3/8                                                                       

 1/61 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.9148 - root_mean_squared_error: 0.9564
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - loss: 0.6886 - root_mean_squared_error: 0.8291 - val_loss: 0.5891 - val_root_mean_squared_error: 0.7675

Epoch 4/8                                

2025/07/14 12:53:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 25/25 [02:09<00:00,  5.16s/trial, best loss: 0.700980007648468]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


2025/07/14 12:53:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Best parameters: {'lr': 0.00925166173166858, 'momentum': 0.9076992431612515}
Best eval rsme: 0.700980007648468
Worst eval rsme: {'loss': 2.2153639793395996, 'status': 'ok', 'model': <Sequential name=sequential_204, built=True>}


In [72]:
print(best_run["model"])


<Sequential name=sequential_194, built=True>
